In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
from scipy.integrate import odeint

import pybel as pb
from pybel.examples import sialic_acid_graph as sag
from pybel.io.jupyter import to_jupyter

import time
import csv

import torch
import pyro
import json
import requests
pyro.set_rng_seed(101)

In [2]:
with open('COVID19.jgf') as f:
    graph_jgif_dict = json.load(f)

In [4]:
graph_jgif_dict

{'graph': {'type': 'Bel',
  'directed': False,
  'label': 'COVID-19',
  'metadata': {'species_id': None,
   'species_label': None,
   'description': None,
   'gd_creator': '',
   'gd_status': 'draft',
   'gd_createTS': '2020-03-25T13:22:02.483Z',
   'gd_updateTS': '2020-03-25T13:24:59.073Z',
   'gd_rev': '_aPFcDD---_',
   'gd_version': '1',
   'gd_description': 'Preliminary Disease Network',
   'id': '01E48XPRM432QH64MMSRA8WA91',
   'schema_uri': 'https://raw.githubusercontent.com/jsongraph/json-graph-specification/master/json-graph-schema.json',
   'gd_immutable': False},
  'nodes': [{'id': 'path(MESH:COVID-19)',
    'label': 'path(MESH:COVID-19)',
    'metadata': {}},
   {'id': 'path(MESH:Hypoalbuminemia)',
    'label': 'path(MESH:Hypoalbuminemia)',
    'metadata': {}},
   {'id': 'p(HGNC:JAK3)', 'label': 'p(HGNC:JAK3)', 'metadata': {}},
   {'id': 'path(MESH:Lymphopenia)',
    'label': 'path(MESH:Lymphopenia)',
    'metadata': {}},
   {'id': 'path(SDIS:"Liver Injury")',
    'label': '

In [3]:
ncount=0
for i in graph_jgif_dict['graph']['nodes']:
    ncount=ncount + 1   
print(ncount)

81


In [4]:
ecount=0
for e in graph_jgif_dict['graph']['edges']:
    ecount=ecount+1
print(ecount)    

113


In [5]:
import re
nodes=[]
parents=[]
children=[]
edgetype=[]
edges=[]
eachedges=[]
nodes_dict=dict()
for eachedge in graph_jgif_dict['graph']['edges']:
    #print(eachedge['id'])
    edges.append(eachedge['id'])
    #eachedges.append(eachedge['id'].split(':')[1])
    s = re.split(r':\s*(?=[^)]*(?:\(|$))', eachedge['id']) 
    eachedges.append(s)
    parents.append(s[0])
    children.append(s[1])
    edgetype.append(s[2])

In [6]:
len(eachedges)

113

In [7]:
df= pd.DataFrame(eachedges)

In [8]:
df.columns=("Parent","Type","Children","uselessNone")

In [9]:
df=df.drop(['uselessNone'],axis=1)

In [111]:
df

,Parent,Type,Children
0,path(MESH:COVID-19),increases,path(MESH:Hypoalbuminemia)
1,p(HGNC:JAK3),decreases,path(MESH:Lymphopenia)
2,"path(SDIS:""Liver Injury"")",increases,p(HGNC:CRP)
3,"bp(GO:""inflammatory response"")",increases,p(HGNC:CRP)
4,p(HGNC:IL6),positiveCorrelation,p(HGNC:CRP)
5,path(SDIS:Diabetes),positiveCorrelation,p(HGNC:CRP)
6,a(SCHEM:fibrate),decreases,p(HGNC:CRP)
7,"path(SDIS:""Chronic Obstructive Pulmonary Disea...",association,p(HGNC:CRP)
8,p(HGNC:TGFB1),increases,p(HGNC:CRP)
9,"path(SDIS:""systemic acute phase reaction"")",increases,p(HGNC:CRP)


In [24]:
parenttype=[]
for i in df['Parent']:
    typestr=i.split('(')
    parenttype.append(typestr[0])
    

In [26]:
df['parent_type']=pd.Series(parenttype)

In [27]:
df.head()

,Parent,Type,Children,parent_type
0,path(MESH:COVID-19),increases,path(MESH:Hypoalbuminemia),path
1,p(HGNC:JAK3),decreases,path(MESH:Lymphopenia),p
2,"path(SDIS:""Liver Injury"")",increases,p(HGNC:CRP),path
3,"bp(GO:""inflammatory response"")",increases,p(HGNC:CRP),bp
4,p(HGNC:IL6),positiveCorrelation,p(HGNC:CRP),p


In [28]:
childrentype=[]
for c in df['Children']:
    childtype=c.split('(')
    childrentype.append(childtype[0])
childrentype

['path',
 'path',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'bp',
 'sec',
 'p',
 'path',
 'path',
 'act',
 'path',
 'tloc',
 'p',
 'p',
 'p',
 'p',
 'act',
 '',
 'path',
 'bp',
 'path',
 'act',
 'bp',
 'p',
 'path',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'act',
 '',
 'bp',
 'p',
 'path',
 'r',
 'path',
 'act',
 'bp',
 'bp',
 'p',
 'bp',
 'bp',
 'bp',
 'bp',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'bp',
 'p',
 'p',
 'p',
 'p',
 'path',
 'r',
 'path',
 'sec',
 'path',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'bp',
 'p',
 'p',
 'p',
 'p',
 'p',
 'pop',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p']

In [29]:
df['child_type']=pd.Series(childrentype)

In [30]:
df=df[["Parent","parent_type","Children","child_type","Type"]]

In [31]:
df.head()

,Parent,parent_type,Children,child_type,Type
0,path(MESH:COVID-19),path,path(MESH:Hypoalbuminemia),path,increases
1,p(HGNC:JAK3),p,path(MESH:Lymphopenia),path,decreases
2,"path(SDIS:""Liver Injury"")",path,p(HGNC:CRP),p,increases
3,"bp(GO:""inflammatory response"")",bp,p(HGNC:CRP),p,increases
4,p(HGNC:IL6),p,p(HGNC:CRP),p,positiveCorrelation


In [32]:
df.tail()

,Parent,parent_type,Children,child_type,Type
108,"act(complex(SCOMP:""AMP Activated Protein Kinas...",act,p(HGNC:ANGPT2),p,decreases
109,"act(p(HGNC:NOD2), ma(cat))",act,p(HGNC:TNF),p,increases
110,path(MESH:COVID-19),path,p(HGNC:CRP),p,increases
111,a(CHEBI:dexamethasone),a,p(HGNC:TNF),p,decreases
112,"act(p(HGNC:MKNK1), ma(kin))",act,p(HGNC:TNF),p,increases
